Overview:
This script performs data processing by cleaning, transforming (downcasting) the data and also merging different datasets for analysis.

Author: Chloe Ang (Ke Er Ang)

# Data Processing

In [1]:
import pandas as pd
import numpy as np

1. Merging sell_price.csv and Calendar.csv to find the price of each item on different day.

In [2]:
selling_price = pd.read_csv('sell_prices.csv')
calendar = pd.read_csv('calendar.csv')

In [5]:
sell_calendar_merge = pd.merge(selling_price, calendar, on='wm_yr_wk')

In [7]:
# combine id and item_id to get id 
sell_calendar_merge['id']= sell_calendar_merge['item_id'] + '_'+sell_calendar_merge['store_id']

In [11]:
sell_calendar_merge.to_csv('sell_calendar.csv')

2. Merging sell_calendar_merge with sales_validation.csv 
    - goals : To find the quantity sold and price of each item on each day
    - separate into 3 parts (Hobbies, Foods, Households)

Step 1: after merging selling_price and calendar, I found that some of the 'd_day' columns have day later than 1913, so we need to filter out.

In [3]:
sell_calendar = pd.read_csv('sell_calendar.csv')

In [6]:
sell_calendar = sell_calendar.drop(sell_calendar.columns[0],axis=1)

In [11]:
# split column 'd' in order to filter out day after 1913
sell_calendar[['prefix', 'number']] = sell_calendar['d'].str.split('_', expand=True)

In [21]:
# drop prefix 'd_' as it is redundant
sell_calendar = sell_calendar.drop(sell_calendar.columns[9],axis=1)

In [24]:
# change column 'number' to type integer
sell_calendar['number'] = sell_calendar['number'].astype(int)

In [26]:
# filter out days that is larger than d_1913
selected_sell_calendar = sell_calendar.loc[sell_calendar['number'] <= 1913]

In [31]:
selected_sell_calendar = selected_sell_calendar.reset_index(drop=True)

In [33]:
# cleaned selling_price + calendar csv
selected_sell_calendar.to_csv('selected_sell_calendar.csv')

In [34]:
selected_sell_calendar 

,id,wm_yr_wk,sell_price,d,year,month,wday,date,amount,number
0,HOBBIES_1_001_CA_1,11325,9.58,d_897,2013,7,1,2013-07-13,0,897
1,HOBBIES_1_001_CA_1,11325,9.58,d_898,2013,7,2,2013-07-14,0,898
2,HOBBIES_1_001_CA_1,11325,9.58,d_899,2013,7,3,2013-07-15,0,899
3,HOBBIES_1_001_CA_1,11325,9.58,d_900,2013,7,4,2013-07-16,0,900
4,HOBBIES_1_001_CA_1,11325,9.58,d_901,2013,7,5,2013-07-17,0,901
...,...,...,...,...,...,...,...,...,...,...
46027952,FOODS_3_825_WI_3,11105,4.00,d_31,2011,2,3,2011-02-28,0,31
46027953,FOODS_3_825_WI_3,11105,4.00,d_32,2011,3,4,2011-03-01,0,32
46027954,FOODS_3_825_WI_3,11105,4.00,d_33,2011,3,5,2011-03-02,0,33
46027955,FOODS_3_825_WI_3,11105,4.00,d_34,2011,3,6,2011-03-03,0,34


Step 2: filter out selected_sell_calendar which id contains only foods or hobbies or households.

In [36]:
selected_sell_calendar_hobbies = selected_sell_calendar[selected_sell_calendar.iloc[:, 0].str.contains('HOBBIES')]

In [38]:
selected_sell_calendar_foods = selected_sell_calendar[selected_sell_calendar.iloc[:, 0].str.contains('FOODS')]

In [39]:
selected_sell_calendar_households  = selected_sell_calendar[selected_sell_calendar.iloc[:, 0].str.contains('HOUSEHOLD')]

Below 3 files are the separated sell_calendar_merge according to the category,
each file contains the price of each item on each day in 5 years.

In [43]:
selected_sell_calendar_hobbies = selected_sell_calendar_hobbies.reset_index(drop=True)

In [46]:
selected_sell_calendar_foods = selected_sell_calendar_foods.reset_index(drop=True)

In [48]:
selected_sell_calendar_households = selected_sell_calendar_households.reset_index(drop=True)

In [50]:
selected_sell_calendar_hobbies.to_csv('selected_sell_calendar_hobbies.csv')

In [51]:
selected_sell_calendar_foods.to_csv('selected_sell_calendar_foods')

In [52]:
selected_sell_calendar_households.to_csv('selected_sell_calendar_households')

Step 3: preprocess sales_train_validation and split into 3 categories

In [4]:
sales_train_validation = pd.read_csv('sales_train_validation.csv')

In [10]:
sales_train_validation

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [11]:
for i in range(len(sales_train_validation['id'])):
    sales_train_validation['id'][i] = sales_train_validation['id'][i].rstrip('_validation')

/var/folders/7h/s4cspbv95x98l3h5nvklx3ym0000gn/T/ipykernel_3298/1021508495.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_train_validation['id'][i] = sales_train_validation['id'][i].rstrip('_validation')


In [12]:
# so now we got sales_train_validation cleaned up
sales_train_validation

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


next, we split sales_train_validation into 3 categories (hobbies, foods, households)

In [49]:
# hobbies category
hobbies_sales = sales_train_validation.loc[sales_train_validation['cat_id'] == 'HOBBIES'].set_index('id')

In [30]:
#foods category
foods_sales = sales_train_validation.loc[sales_train_validation['cat_id'] == 'FOODS'].set_index('id')

In [14]:
households_sales = sales_train_validation.loc[sales_train_validation['cat_id'] == 'HOUSEHOLD'].set_index('id')

In [15]:
#households category
households_sales

,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
id,,,,,,,,,,,,,,,,,,,,,
HOUSEHOLD_1_001_CA_1,HOUSEHOLD_1_001,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,3,0,0,0,0,...,1,0,1,1,1,0,0,0,0,1
HOUSEHOLD_1_002_CA_1,HOUSEHOLD_1_002,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,1,1,2,4,1,...,0,0,0,0,0,1,0,0,0,1
HOUSEHOLD_1_003_CA_1,HOUSEHOLD_1_003,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,0,...,0,0,1,2,1,0,0,0,1,0
HOUSEHOLD_1_004_CA_1,HOUSEHOLD_1_004,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,4,0,8,11,1,...,0,5,0,0,0,0,0,0,1,0
HOUSEHOLD_1_005_CA_1,HOUSEHOLD_1_005,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,3,2,2,1,...,1,0,3,1,2,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOUSEHOLD_2_512_WI_3,HOUSEHOLD_2_512,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,0,2,0,0,0,...,0,0,0,2,0,1,0,0,0,0
HOUSEHOLD_2_513_WI_3,HOUSEHOLD_2_513,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
HOUSEHOLD_2_514_WI_3,HOUSEHOLD_2_514,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0


Step 4: we want to combine the dataframe in order to get the quantity sold and price of each item on each day in 5 years.

 - combining hobbies sales & selected_sell_calendar_hobbies

In [71]:
for i in range(len(selected_sell_calendar_hobbies['id'])):
    selected_sell_calendar_hobbies.at[i, 'amount'] = hobbies_sales.loc[selected_sell_calendar_hobbies['id'][i],selected_sell_calendar_hobbies['d'][i]]

In [23]:
for i in range(len(selected_sell_calendar_households['id'])):
    selected_sell_calendar_households.at[i, 'amount'] = households_sales.loc[selected_sell_calendar_households['id'][i],selected_sell_calendar_households['d'][i]]

In [31]:
for i in range(len(selected_sell_calendar_foods['id'])):
    selected_sell_calendar_foods.at[i, 'amount'] = foods_sales.loc[selected_sell_calendar_foods['id'][i],selected_sell_calendar_foods['d'][i]]

In [74]:
selected_sell_calendar_hobbies.to_csv('final_sell_calendar_hobbies.csv')

In [26]:
selected_sell_calendar_households.to_csv('final_sell_calendar_households.csv')

In [32]:
selected_sell_calendar_foods.to_csv('final_sell_calendar_foods.csv')

In [41]:
sales_train_validation[sales_train_validation['item_id']=='HOBBIES_1_001']

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
3049,HOBBIES_1_001_CA_2,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_2,CA,0,0,0,0,...,0,0,1,2,0,4,0,0,2,2
6098,HOBBIES_1_001_CA_3,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_3,CA,0,0,0,0,...,0,2,4,0,1,1,1,0,3,3
9147,HOBBIES_1_001_CA_4,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_4,CA,0,0,0,0,...,1,0,0,1,0,1,0,1,2,1
12196,HOBBIES_1_001_TX_1,HOBBIES_1_001,HOBBIES_1,HOBBIES,TX_1,TX,0,0,0,0,...,0,0,0,0,1,0,1,0,0,2
15245,HOBBIES_1_001_TX_2,HOBBIES_1_001,HOBBIES_1,HOBBIES,TX_2,TX,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
18294,HOBBIES_1_001_TX_3,HOBBIES_1_001,HOBBIES_1,HOBBIES,TX_3,TX,0,0,0,0,...,0,3,0,0,0,0,0,0,0,0
21343,HOBBIES_1_001_WI_1,HOBBIES_1_001,HOBBIES_1,HOBBIES,WI_1,WI,0,0,0,0,...,0,0,0,0,0,0,1,2,1,1
24392,HOBBIES_1_001_WI_2,HOBBIES_1_001,HOBBIES_1,HOBBIES,WI_2,WI,0,0,0,0,...,1,1,0,0,0,0,0,0,0,1
27441,HOBBIES_1_001_WI_3,HOBBIES_1_001,HOBBIES_1,HOBBIES,WI_3,WI,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [42]:
sales_train_validation[sales_train_validation['item_id']=='HOBBIES_2_001']

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
416,HOBBIES_2_001_CA_1,HOBBIES_2_001,HOBBIES_2,HOBBIES,CA_1,CA,1,0,0,0,...,0,2,0,0,0,0,0,0,1,0
3465,HOBBIES_2_001_CA_2,HOBBIES_2_001,HOBBIES_2,HOBBIES,CA_2,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6514,HOBBIES_2_001_CA_3,HOBBIES_2_001,HOBBIES_2,HOBBIES,CA_3,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9563,HOBBIES_2_001_CA_4,HOBBIES_2_001,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
12612,HOBBIES_2_001_TX_1,HOBBIES_2_001,HOBBIES_2,HOBBIES,TX_1,TX,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15661,HOBBIES_2_001_TX_2,HOBBIES_2_001,HOBBIES_2,HOBBIES,TX_2,TX,0,0,0,0,...,0,0,1,1,0,0,1,0,0,0
18710,HOBBIES_2_001_TX_3,HOBBIES_2_001,HOBBIES_2,HOBBIES,TX_3,TX,0,1,0,0,...,0,0,2,0,0,0,2,0,0,0
21759,HOBBIES_2_001_WI_1,HOBBIES_2_001,HOBBIES_2,HOBBIES,WI_1,WI,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
24808,HOBBIES_2_001_WI_2,HOBBIES_2_001,HOBBIES_2,HOBBIES,WI_2,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27857,HOBBIES_2_001_WI_3,HOBBIES_2_001,HOBBIES_2,HOBBIES,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
hobbies_sales

,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
id,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_001_CA_1,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
HOBBIES_1_002_CA_1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
HOBBIES_1_003_CA_1,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
HOBBIES_1_004_CA_1,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
HOBBIES_1_005_CA_1,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOBBIES_2_145_WI_3,HOBBIES_2_145,HOBBIES_2,HOBBIES,WI_3,WI,0,0,0,0,0,...,0,0,0,0,0,0,4,0,0,0
HOBBIES_2_146_WI_3,HOBBIES_2_146,HOBBIES_2,HOBBIES,WI_3,WI,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
HOBBIES_2_147_WI_3,HOBBIES_2_147,HOBBIES_2,HOBBIES,WI_3,WI,0,0,0,0,0,...,1,0,0,0,0,2,0,1,2,0


In [51]:
sales = pd.read_csv('sales_train_validation.csv')
calendar = pd.read_csv('calendar.csv')
prices = pd.read_csv('sell_prices.csv')

In [7]:
#Downcast in order to save memory
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == np.object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df  

sales = downcast(sales_train_validation)
prices = downcast(selling_price)
calendar = downcast(calendar)

/var/folders/7h/s4cspbv95x98l3h5nvklx3ym0000gn/T/ipykernel_957/3849706072.py:22: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif t == np.object:


In [10]:
sales.to_csv('sales.csv')
prices.to_csv('prices.csv')
calendar.to_csv('calendar2.csv')

In [11]:
df = pd.melt(sales, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='sold').dropna()

In [12]:
df = pd.merge(df, calendar, on='d', how='left')

In [13]:
df = pd.merge(df, prices, on=['store_id','item_id','wm_yr_wk'], how='left') 

In [15]:
df.to_csv('master.csv')

KeyboardInterrupt: 

In [17]:
hobbies_df = df[df['cat_id'] =='HOBBIES']

In [27]:
hobbies_df

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58324881,HOBBIES_2_145_WI_3_validation,HOBBIES_2_145,HOBBIES_2,HOBBIES,WI_3,WI,d_1913,0,2016-04-24,11613,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,2.880859
58324882,HOBBIES_2_146_WI_3_validation,HOBBIES_2_146,HOBBIES_2,HOBBIES,WI_3,WI,d_1913,0,2016-04-24,11613,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,1.969727
58324883,HOBBIES_2_147_WI_3_validation,HOBBIES_2_147,HOBBIES_2,HOBBIES,WI_3,WI,d_1913,0,2016-04-24,11613,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,0.970215
58324884,HOBBIES_2_148_WI_3_validation,HOBBIES_2_148,HOBBIES_2,HOBBIES,WI_3,WI,d_1913,0,2016-04-24,11613,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,0.879883


In [19]:
foods_df = df[df['cat_id'] =='FOODS']

In [24]:
households_df = df[df['cat_id'] =='HOUSEHOLD']

In [21]:
foods_df.to_csv('foods.csv')

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
1612,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1,3,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,2.000000
1613,FOODS_1_002_CA_1_validation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,7.878906
1614,FOODS_1_003_CA_1_validation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,2.880859
1615,FOODS_1_004_CA_1_validation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
1616,FOODS_1_005_CA_1_validation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,d_1,3,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,2.939453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58327365,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1913,1,2016-04-24,11613,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,2.980469
58327366,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1913,0,2016-04-24,11613,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,2.480469
58327367,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1913,0,2016-04-24,11613,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,3.980469
58327368,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1913,3,2016-04-24,11613,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,1.280273


In [25]:
households_df.to_csv('households.csv')

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
565,HOUSEHOLD_1_001_CA_1_validation,HOUSEHOLD_1_001,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,d_1,3,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,6.320312
566,HOUSEHOLD_1_002_CA_1_validation,HOUSEHOLD_1_002,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,d_1,1,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,6.320312
567,HOUSEHOLD_1_003_CA_1_validation,HOUSEHOLD_1_003,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
568,HOUSEHOLD_1_004_CA_1_validation,HOUSEHOLD_1_004,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,d_1,4,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,1.980469
569,HOUSEHOLD_1_005_CA_1_validation,HOUSEHOLD_1_005,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,10.718750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58325928,HOUSEHOLD_2_512_WI_3_validation,HOUSEHOLD_2_512,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,d_1913,0,2016-04-24,11613,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,3.970703
58325929,HOUSEHOLD_2_513_WI_3_validation,HOUSEHOLD_2_513,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,d_1913,0,2016-04-24,11613,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,2.779297
58325930,HOUSEHOLD_2_514_WI_3_validation,HOUSEHOLD_2_514,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,d_1913,0,2016-04-24,11613,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,19.546875
58325931,HOUSEHOLD_2_515_WI_3_validation,HOUSEHOLD_2_515,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,d_1913,0,2016-04-24,11613,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,1.969727


In [37]:
foods_df2 = foods_df[['id','item_id','dept_id','cat_id','store_id','date','month','year','wm_yr_wk','sell_price','sold']]

In [38]:
foods_df2.to_csv('final_sell_calendar_foods2.csv')